<a href="https://colab.research.google.com/github/rogerioag/rea-comp04-compiladores/blob/main/jupyter-notebooks/01-comp-analise-lexica-cmmlex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise Léxica

A __Análise Léxica__ é a fase do compilador que lê o código-fonte do arquivo de entrada como um fluxo de caracteres, e nesse processo de varredura reconhece os _tokens_ ou marcas da linguagem. As denominações Sistema de Varredura, Analisador Léxico e _Scanner_ são equivalentes.



## Introdução

Devem ser reconhecidas as marcas presentes na linguagem `C-`, como `if`, `while` que são palavras chave, palavras reservadas. Precisam ser reconhecidos os nomes de variáveis e funções que são os _identificadores_, símbolos e operadores aritméticos, lógicos e relacionais.

O processo de reconhecimento das marcas, a identificação de padrões pode ser feito de duas formas: utilizando-se __expressões regulares__ ou implementando o analisador com a teoria de __autômatos finitos__.

Neste tutorial iremos utilizar _expressões regulares_ para a especificarmos os padrões de reconhecimentos das marcas.

Para um código simples em `C-` igual o Código 1.

```c
/* Programa simples. */

int main(void){
  return(0);
}
```
_Código 1: Programa em C-_


A lista de marcas que precisam ser identificadas é:
```
INT
ID
LPAREN
VOID
RPAREN
LBRACES
RETURN
LPAREN
NUMBER
RPAREN
SEMICOLON
RBRACES
```

## Leitura Recomendada

1. __Capítulo 2:__ _Varredura_

    LOUDEN, Kenneth C. Compiladores: princípios e práticas. São Paulo, SP: Thomson, c2004. xiv, 569 p. ISBN 8522104220.

2. __Capítulo 3:__ _Análise Léxica_

    AHO, Alfred V. et al. Compiladores: princípios, técnicas e ferramentas. 2. ed. São Paulo, SP: Pearson Addison-Wesley, 2008. x, 634 p. ISBN 9788588639249.

## Preparação do Ambiente

Para a implementação da Análise Léxica é necessário instalar a ferramentas `PLY`:

In [ ]:
!pip install ply
!pip install anytree
!pip install graphviz
!pip install llvmlite
!jupyter nbextension install https://rawgit.com/jfbercher/small_nbextensions/master/highlighter.zip  --user
!jupyter nbextension enable highlighter/highlighter

Downloading: https://rawgit.com/jfbercher/small_nbextensions/master/highlighter.zip -> /tmp/tmprfYPBo/highlighter.zip
Extracting: /tmp/tmprfYPBo/highlighter.zip -> /root/.local/share/jupyter/nbextensions
Enabling notebook extension highlighter/highlighter...
      - Validating: OK


In [ ]:
%%javascript
require("base/js/utils").load_extensions("highlighter/highlighter")

<IPython.core.display.Javascript object>

## Copiando Módulos do Projeto do Github

Para a realização de testes com o código de referência (código de _start_) podemos baixar os arquivos do projeto `rea-comp04-compiladores` do repositório do [`github`](https://github.com/rogerioag/rea-comp04-compiladores):

```bash
! git clone https://github.com/rogerioag/rea-comp04-compiladores.git
```
Comandos do `terminal` podem ser executados sendo precedidos de `!`.

Iremos copiar os módulos e os arquivos de testes para o diretório de conteúdo do _notebook_, o __content__.


In [ ]:
! rm -rf rea-comp04-compiladores
! git clone https://github.com/rogerioag/rea-comp04-compiladores.git
! cp -R rea-comp04-compiladores/cmmcompiler/* .
! cp -R rea-comp04-compiladores/cmmcompiler/tests/* .


Cloning into 'rea-comp04-compiladores'...
remote: Enumerating objects: 474, done.
remote: Counting objects: 100% (474/474), done.
remote: Compressing objects: 100% (378/378), done.
remote: Total 474 (delta 268), reused 228 (delta 85), pack-reused 0
Receiving objects: 100% (474/474), 2.87 MiB | 4.28 MiB/s, done.
Resolving deltas: 100% (268/268), done.


## Código de Referência

O código do projeto `cmmcompiler` foi copiado para o servidor do `Colab`.
Utilizando `%%writefile` é possível gravar o conteúdo da célula do _notebook_ em um arquivo do lado servidor do `Colab`, inclusive alterar o conteúdo original dos arquivos do projeto.

O arquivo `lexer/tokens.py` faz definição das palavras reservadas, símbolos matemáticos como operadores aritméticos, operadores relacionais, símbolos de controle, identificadores, e da lista de _tokens_: 

In [ ]:
%%writefile lexer/tokens.py

__all__ = ['tokens', 'TOKENS_SYMBOLS']

reserved = {
    'else': 'ELSE',
    'if': 'IF',
    'int': 'INT',
    'return': 'RETURN',
    'void': 'VOID',
    'while': 'WHILE',
}

math_symbols = [
    'PLUS',  # +
    'MINUS',  # -
    'TIMES',  # *
    'DIVIDE'  # /
]

comparison_symbols = [
    'LESS_EQUAL',  # <=
    'LESS',  # <
    'GREATER_EQUAL',  # >=
    'GREATER',  # >
    'EQUALS',  # ==
    'DIFFERENT',  # !=
]

control_symbols = [
    'LPAREN',  # (
    'RPAREN',  # )
    'LBRACKETS',  # [
    'RBRACKETS',  # ]
    'LBRACES',  # {
    'RBRACES',  # }
    'ATTRIBUTION',  # =
    'SEMICOLON',  # ;
    'COMMA',  # ,
]

markers = [
    'ID',
    'NUMBER',
]

TOKENS_SYMBOLS = {
    'PLUS': '+',
    'MINUS': '-',
    'TIMES': '*',
    'DIVIDE': '/',
    'LESS_EQUAL': '<=',
    'LESS': '<',
    'GREATER_EQUAL': '>=',
    'GREATER': '>',
    'EQUALS': '==',
    'DIFFERENT': '!=',
    'LPAREN': '(',
    'RPAREN': ')',
    'LBRACKETS': '[',
    'RBRACKETS': ']',
    'LBRACES': '{',
    'RBRACES': '}',
    'ATTRIBUTION': '=',
    'SEMICOLON': ';',
    'COMMA': ',',
    'ELSE': 'else',
    'IF': 'if',
    'INT': 'int',
    'RETURN': 'return',
    'VOID': 'void',
    'WHILE': 'while',
}

tokens = markers + math_symbols + comparison_symbols + \
    control_symbols + list(reserved.values())


Overwriting lexer/tokens.py


Definição das Expressões Regulares para cada uma das classes de _tokens_: `lexer/regexs.py`

In [ ]:
%%writefile lexer/regexs.py

id_regex = r'[a-zA-Z][a-zA-Z]*'
comment_regex = r'\/\*[^\r]*\*\/'

#! MATH
t_PLUS = r'\+'
t_MINUS = r'-'
t_TIMES = r'\*'
t_DIVIDE = r'/'

#! COMPARISON
t_LESS_EQUAL = r'<='
t_LESS = r'<'
t_GREATER_EQUAL = r'>='
t_GREATER = r'>'
t_EQUALS = r'=='
t_DIFFERENT = r'!='

#! CONTROL
t_LPAREN = r'\('
t_RPAREN = r'\)'
t_LBRACKETS = r'\['
t_RBRACKETS = r'\]'
t_LBRACES = r'{'
t_RBRACES = r'}'
t_ATTRIBUTION = r'='
t_SEMICOLON = r';'
t_COMMA = r','

t_NUMBER = r'[0-9][0-9]*'


Overwriting lexer/regexs.py


Definição das funções que representam as ações para cada classe de _token_ reconhecida: `lexer/methods.py`

In [ ]:
%%writefile lexer/methods.py

from ply.lex import TOKEN

from .regexs import id_regex, comment_regex
from .tokens import reserved

t_ignore = ' \t'

@TOKEN(id_regex)
def t_ID(t):
    t.type = reserved.get(t.value, 'ID')
    return t

@TOKEN(comment_regex)
def t_ignore_COMMENT(r):
    pass

def t_newline(t):
    r'\n+'
    t.lexer.lineno += len(t.value)

def t_error(t):
    print("Símbolo não definido pela linguagem '%s'" % t.value[0])
    t.lexer.skip(1)

Overwriting lexer/methods.py


Código principal do módulo `lexer`, instancia o `lexer` e define a função que devolve uma lista de _tokens_: `lexer/__init__.py`

In [ ]:
%%writefile lexer/__init__.py

__all__ = ['tokens', 'lexer', 'get_tokens', 'TOKENS_SYMBOLS', 'number_regex']

# ------------------------------------------------------------
# tokenizer for a C subset language called C-
# ------------------------------------------------------------
import ply.lex as lex
from ply.lex import TOKEN

from .tokens import *
from .regexs import *
from .methods import *

from .regexs import t_NUMBER as number_regex

from sys import argv, exit

import logging

logging.basicConfig(
     level = logging.DEBUG,
     filename = "cmmcompiler.log",
     filemode = "w",
     format = "%(filename)10s:%(lineno)4d:%(message)s"
)
log = logging.getLogger()

def get_tokens(input):
    lexer = lex.lex()
    lexer.input(input)

    tokens = []
    token = lexer.token()
    while token:
        tokens.append(token)
        token = lexer.token()
        pass

    return tokens
    pass


lexer = lex.lex(optimize=True,debug=True,debuglog=log)


Overwriting lexer/__init__.py


Código principal `cmmcompiler` que executa o módulo `lexer` e as outras fases do compilador: `main.py`.

Nesta fase de Compilação será utilizada a função:

```python
def execute_lexical_analysis(source_input):
    log.info("[lexer]: Executing Lexical Analysis.")
    for token in get_tokens(source_input):
        print(token.type, token.value)
    return
```

Esta função será chamada quando utilizarmos a opção `-l`com o `main.py`.

In [ ]:
%%writefile main.py

import utils
from lexer import get_tokens
from parser import parser
from semantic import sema, Semantic
from gencode import gencode, GenCode
from tree import TreeNode

import logging
logging.basicConfig(
    filename = "cmmcompiler.log",
    encoding='utf-8',
    level = logging.DEBUG,
    filemode = "w",
    format = "%(filename)10s:%(lineno)4d:%(message)s")
log = logging.getLogger()

syntax_tree = None
reduced_syntax_tree = None

def execute_lexical_analysis(source_input):
    log.info("[lexer]: Executing Lexical Analysis.")
    for token in get_tokens(source_input):
        print(token.type, token.value)
    return

def execute_syntax_analisys(source_input):
    log.info("[parser]: Executing Syntax Analysis.")
    syntax_tree = parser.parse(source_input)
    if syntax_tree != ():
        print("Generating Syntax Tree Graph...")
        graph = utils.Graph(utils.args.file, 'Sintax Tree')
        # program = parser.parse(source_input)
        syntax_tree.render(graph)
        graph.export()
    return syntax_tree

def execute_semantic_analisys(syntax_tree):
    log.info("[sema]: Executing Semantic Analysis.")
    sema = Semantic(syntax_tree)
    sema.check_semantic_rules()
    return reduced_syntax_tree

def execute_code_generation(reduced_syntax_tree):
    log.info("[gencode]: Executing Code Generation.")
    gencode = GenCode(reduced_syntax_tree)
    gencode.generate()
    return

if __name__ == '__main__':

  with open(utils.args.file) as file:
      source_input = file.read()
      if utils.args.lexer:
          execute_lexical_analysis(source_input)    
      pass

      if utils.args.parser:
          syntax_tree = execute_syntax_analisys(source_input) 
      pass
        
      if utils.args.semantic:
          if syntax_tree != ():
              reduced_syntax_tree = execute_semantic_analisys(syntax_tree)
          else:
              syntax_tree = execute_syntax_analisys(source_input)
              reduced_syntax_tree = execute_semantic_analisys(syntax_tree)
          pass
      pass

      if utils.args.gencode:
          if reduced_syntax_tree != None:
              execute_code_generation(reduced_syntax_tree)
          else:
              syntax_tree = execute_syntax_analisys(source_input)
              if syntax_tree != ():
                  reduced_syntax_tree = execute_semantic_analisys(syntax_tree)
                  execute_code_generation(reduced_syntax_tree)
              pass

      pass
        
  pass


Overwriting main.py


## Testes de outros exemplos

Alguns arquivos de testes foram definidos no projeto. Utilizando `%%writefile` é possível gravar o conteúdo da célula do _notebook_ em um arquivo do lado servidor do Colab:

In [ ]:
%%writefile prog-001.cm

int main(){
}

Overwriting prog-001.cm


In [ ]:
%%writefile prog-002.cm

int main(){
    return 0;
}

Writing prog-002.cm


A implementação do _Analisador Léxico_ pode ser chamada a partir do código principal `main.py` com o parâmetro `-l`: 

In [ ]:
! python main.py -l prog-001.cm

Generating LALR tables
INT int
ID main
LPAREN (
VOID void
RPAREN )
LBRACES {
RBRACES }


## Projeto de Implementação

A gramática da linguagem `C-` tem suporte somente para os tipos `int` e `void`. Como Trabalho de Implementação vinculado à _Análise Léxica_, além da implementação do `Analisador Léxico` pode ser solicitado aos alunos que implementem o suporte ao tipo `float`, para que seja possível trabalhar com variáveis do tipo ponto flutuante.

Que se daria pela inclusão de _float_ na lista de palavras _reservadas_:

```c
reserved = {
    'else': 'ELSE',
    'if': 'IF',
    'int': 'INT',
    'float': 'FLOAT', // Adição do float.
    'return': 'RETURN',
    'void': 'VOID',
    'while': 'WHILE',
}
```

E pela adição do _token_ para _float_ na lista de _tokens_:

```c
TOKENS_SYMBOLS = {
    'PLUS': '+',
    'MINUS': '-',
    'TIMES': '*',
    'DIVIDE': '/',
    'LESS_EQUAL': '<=',
    'LESS': '<',
    'GREATER_EQUAL': '>=',
    'GREATER': '>',
    'EQUALS': '==',
    'DIFFERENT': '!=',
    'LPAREN': '(',
    'RPAREN': ')',
    'LBRACKETS': '[',
    'RBRACKETS': ']',
    'LBRACES': '{',
    'RBRACES': '}',
    'ATTRIBUTION': '=',
    'SEMICOLON': ';',
    'COMMA': ',',
    'ELSE': 'else',
    'IF': 'if',
    'INT': 'int',
    'FLOAT': 'float', // Adição do Token FLOAT.
    'RETURN': 'return',
    'VOID': 'void',
    'WHILE': 'while',
}
```


## Referências

[1] LOUDEN, Kenneth C. **Compiladores:** princípios e práticas. São Paulo, SP: Thomson, c2004. xiv, 569 p. ISBN 8522104220.

[2] AHO, Alfred V. **Compiladores:** princípios, técnicas e ferramentas. 2. ed. São Paulo: Pearson Addison-Wesley, 2008. x, 634 p. ISBN 9788588639249.